In [2]:
%reload_ext ishbook
#%use pandas==0.19.2
%use pandas==0.22.0
import pandas as pd
import numpy as np
import os
import getpass
import pg8000
import MySQLdb
from contrib.fetch import sanitize_for_query
import plus
import datetime
import json
from IPython.display import display, HTML

ERROR: Line magic function `%use` not found.


In [46]:
# To get txt data in dataframe
import json
from pandas.io.json import json_normalize #package for flattening json in pandas df

def txt_to_df(txt_file_loc):
    txt =  open(txt_file_loc,"r").read()
    txt = json.loads(txt)
    df = json_normalize(txt['results'])
    return df

fulltime_df = txt_to_df("job_ad_fulltime_with_summary.txt")
parttime_df = txt_to_df("job_ad_parttime_with_summary.txt")

In [47]:
fulltime_df['hours']='fulltime'
parttime_df['hours'] = 'parttime'

In [48]:
pay_df = pd.read_csv('UK Gender Pay Gap Data - 2017 to 2018.csv')

In [49]:
df = pd.concat([fulltime_df,parttime_df])

In [50]:
import difflib
# matching company names between pay data and job description data
df['Matches'] = df['company'].map(lambda x: difflib.get_close_matches(x, pay_df['EmployerName']))
match_df = df.Matches.apply(pd.Series).rename(columns={0:'EmployerName'})
df = pd.concat([match_df['EmployerName'],df], axis=1)
pay_desc_df = pd.merge(pay_df,df, how="inner", on='EmployerName')

In [92]:
searchfor = ['equal', 'diversity', 'equality of opportunit']
pay_desc_df['diversity statement'] = pay_desc_df['jobsummary'].str.contains('|'.join(searchfor))

In [97]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

pay_desc_df["jobsummary_clean"] = pay_desc_df.jobsummary.apply(lambda x: cleanhtml(x))

In [99]:
diversity_job_summary = pay_desc_df[pay_desc_df['diversity statement']]['jobsummary_clean']
diversity_job_summary.to_csv('diversity_job_summary.csv', header=True, index=False, encoding='utf-8')

#non diverse job summaries (visually inspect)

nondiversity_job_summary = pay_desc_df[~pay_desc_df['diversity statement']]['jobsummary_clean']
nondiversity_job_summary.to_csv('nondiversity_job_summary.csv', header=True, index=False, encoding='utf-8')
